# Comprehensive Error Analysis

This notebook performs deep error analysis on the optimized ensemble models to understand:
1. Where models fail and why
2. Patterns in misclassifications
3. Insights for CodeBERT strategy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import joblib
import sys
sys.path.append('..')

from src.models.data_loader import DataLoader

# Load data
loader = DataLoader()
train_df, val_df, test_df = loader.load_data()

code_smells = [
    'has_long_method',
    'has_high_complexity',
    'has_too_many_params',
    'has_deep_nesting',
    'has_no_docstring'
]

## Section 1: Confusion Matrix Analysis

In [ ]:
print("SECTION 1: CONFUSION MATRIX DEEP DIVE")
print("="*70)

confusion_results = []

for smell in code_smells:
    print(f"\n{'='*70}")
    print(f"Analyzing: {smell}")
    print(f"{'='*70}")
    
    # Load best model
    model = joblib.load(f'../models/ensemble/{smell}_voting.pkl')
    
    # Get predictions
    X_val, _, _, y_val, _, _ = loader.prepare_data_for_smell(
        train_df, val_df, test_df, smell
    )
    
    y_pred = model.predict(X_val)
    
    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate rates
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Positive Rate
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0  # False Negative Rate
    
    print(f"\nConfusion Matrix:")
    print(f"  True Negatives:  {tn:4d} (correctly identified clean code)")
    print(f"  False Positives: {fp:4d} (clean code flagged as issue) - FPR: {fpr:.2%}")
    print(f"  False Negatives: {fn:4d} (missed real issues) - FNR: {fnr:.2%}")
    print(f"  True Positives:  {tp:4d} (correctly caught issues)")
    
    # Which is worse?
    if fpr > fnr:
        error_type = "High False Positive Rate: Model over-flags clean code"
    elif fnr > fpr:
        error_type = "High False Negative Rate: Model misses real issues"
    else:
        error_type = "Balanced error rates"
    
    print(f"\n⚠️ {error_type}")
    
    confusion_results.append({
        'smell': smell,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'fpr': fpr,
        'fnr': fnr,
        'error_type': error_type
    })

confusion_df = pd.DataFrame(confusion_results)
confusion_df

## Section 2: Misclassification Patterns

In [ ]:
print("\nSECTION 2: ANALYZING MISCLASSIFIED SAMPLES")
print("="*70)

def analyze_misclassifications(smell, model, X_val, y_val, val_df_subset, n_samples=3):
    """Analyze specific examples of errors"""
    
    y_pred = model.predict(X_val)
    
    # Get indices of errors
    fp_indices = np.where((y_val == 0) & (y_pred == 1))[0]  # False Positives
    fn_indices = np.where((y_val == 1) & (y_pred == 0))[0]  # False Negatives
    
    print(f"\n{smell}")
    print("-"*70)
    print(f"False Positives: {len(fp_indices)} samples")
    print(f"False Negatives: {len(fn_indices)} samples")
    
    results = {
        'smell': smell,
        'fp_count': len(fp_indices),
        'fn_count': len(fn_indices),
        'fp_examples': [],
        'fn_examples': []
    }
    
    # Analyze False Positives
    if len(fp_indices) > 0:
        print(f"\nSample False Positives (predicted issue, but actually clean):")
        for i, idx in enumerate(fp_indices[:n_samples]):
            if idx < len(val_df_subset):
                code = val_df_subset.iloc[idx]['code']
                print(f"\nFP #{i+1}:")
                print(f"  Lines: {len(code.split(chr(10)))}")
                print(f"  Code snippet: {code[:150]}...")
                results['fp_examples'].append(code[:150])
    
    # Analyze False Negatives
    if len(fn_indices) > 0:
        print(f"\nSample False Negatives (real issue, but not detected):")
        for i, idx in enumerate(fn_indices[:n_samples]):
            if idx < len(val_df_subset):
                code = val_df_subset.iloc[idx]['code']
                print(f"\nFN #{i+1}:")
                print(f"  Lines: {len(code.split(chr(10)))}")
                print(f"  Code snippet: {code[:150]}...")
                results['fn_examples'].append(code[:150])
    
    return results

# Run analysis for each smell
misclass_results = []
for smell in code_smells:
    model = joblib.load(f'../models/ensemble/{smell}_voting.pkl')
    X_val, _, _, y_val, _, _ = loader.prepare_data_for_smell(
        train_df, val_df, test_df, smell
    )
    
    result = analyze_misclassifications(smell, model, X_val, y_val, val_df, n_samples=3)
    misclass_results.append(result)

## Section 3: Feature Patterns in Errors

In [ ]:
print("\nSECTION 3: FEATURE PATTERNS IN ERRORS")
print("="*70)

feature_analysis = []

for smell in code_smells:
    model = joblib.load(f'../models/ensemble/{smell}_voting.pkl')
    X_val, _, _, y_val, _, _ = loader.prepare_data_for_smell(
        train_df, val_df, test_df, smell
    )
    
    y_pred = model.predict(X_val)
    
    # Compare features: correct vs incorrect predictions
    correct_mask = (y_val == y_pred)
    incorrect_mask = ~correct_mask
    
    print(f"\n{smell}:")
    print("-"*70)
    
    feature_names = loader.get_feature_names()
    
    # Key features for this smell
    if smell == 'has_long_method':
        key_features = ['num_lines', 'cyclomatic_complexity', 'num_function_calls']
    elif smell == 'has_high_complexity':
        key_features = ['cyclomatic_complexity', 'num_if_statements']
    elif smell == 'has_too_many_params':
        key_features = ['num_parameters', 'num_default_args']
    elif smell == 'has_deep_nesting':
        key_features = ['num_if_statements', 'num_for_loops']
    else:  # has_no_docstring
        key_features = ['num_comments', 'num_lines']
    
    print(f"Key feature comparison (correct vs incorrect predictions):")
    for feat in key_features:
        if feat in feature_names:
            feat_idx = feature_names.index(feat)
            
            if correct_mask.sum() > 0:
                correct_mean = X_val[correct_mask, feat_idx].mean()
            else:
                correct_mean = 0
            
            if incorrect_mask.sum() > 0:
                incorrect_mean = X_val[incorrect_mask, feat_idx].mean()
            else:
                incorrect_mean = 0
            
            print(f"  {feat:30s}: Correct={correct_mean:8.2f}, Incorrect={incorrect_mean:8.2f}")
            
            feature_analysis.append({
                'smell': smell,
                'feature': feat,
                'correct_mean': correct_mean,
                'incorrect_mean': incorrect_mean
            })

## Section 4: Boundary Cases

In [ ]:
print("\nSECTION 4: BOUNDARY CASES")
print("="*70)

boundary_results = []

for smell in code_smells:
    model = joblib.load(f'../models/ensemble/{smell}_voting.pkl')
    X_val, _, _, y_val, _, _ = loader.prepare_data_for_smell(
        train_df, val_df, test_df, smell
    )
    
    # Get prediction probabilities
    y_proba = model.predict_proba(X_val)[:, 1]  # Probability of positive class
    
    # Find uncertain predictions (probability near 0.5)
    uncertain_mask = (y_proba > 0.4) & (y_proba < 0.6)
    n_uncertain = uncertain_mask.sum()
    
    print(f"\n{smell}:")
    print(f"  Uncertain predictions (40-60% confidence): {n_uncertain} ({n_uncertain/len(y_val)*100:.1f}%)")
    
    # Accuracy on uncertain samples
    if n_uncertain > 0:
        y_pred_uncertain = (y_proba[uncertain_mask] > 0.5).astype(int)
        y_true_uncertain = y_val[uncertain_mask]
        
        uncertain_accuracy = (y_pred_uncertain == y_true_uncertain).mean()
        print(f"  Accuracy on uncertain samples: {uncertain_accuracy:.2%}")
        print(f"  → Model struggles with borderline cases")
        
        boundary_results.append({
            'smell': smell,
            'uncertain_count': n_uncertain,
            'uncertain_pct': n_uncertain/len(y_val)*100,
            'uncertain_accuracy': uncertain_accuracy
        })
    else:
        boundary_results.append({
            'smell': smell,
            'uncertain_count': 0,
            'uncertain_pct': 0,
            'uncertain_accuracy': 0
        })

boundary_df = pd.DataFrame(boundary_results)
boundary_df

## Section 5: Key Insights

In [ ]:
print("\nSECTION 5: KEY INSIGHTS FROM ERROR ANALYSIS")
print("="*70)

insights = {
    'has_long_method': [
        "Perfect performance due to direct feature (num_lines)",
        "No errors to analyze - trivial classification task"
    ],
    'has_high_complexity': [
        "Perfect performance due to cyclomatic_complexity feature",
        "Feature-based approach is sufficient for this smell"
    ],
    'has_too_many_params': [
        "Perfect performance - num_parameters is direct indicator",
        "Clear threshold-based detection"
    ],
    'has_deep_nesting': [
        "Perfect performance with nesting depth features",
        "Static analysis metrics are highly effective"
    ],
    'has_no_docstring': [
        "Most challenging task (80.73% F1)",
        "Errors occur on borderline cases",
        "One-liner functions often lack docstrings (acceptable?)",
        "Model can't distinguish required vs optional docs",
        "Some functions are self-documenting (name + types)",
        "→ CodeBERT's semantic understanding should help here"
    ]
}

for smell, smell_insights in insights.items():
    print(f"\n{smell}:")
    for insight in smell_insights:
        print(f"  • {insight}")

print("\n\nOVERALL LEARNINGS:")
print("="*70)
print("""
1. Feature-based models achieve perfect scores on structural smells
   → This is expected when features directly encode the labeling logic

2. "No Docstring" is the only non-trivial task (80.73% F1)
   → This is where CodeBERT can add value through semantic understanding

3. Borderline cases are inherently ambiguous
   → Human judgment varies - 80% may be ceiling for some smells

4. Automated metrics don't capture all aspects of quality
   → Need semantic understanding of code intent and context

5. CodeBERT Strategy:
   → Focus on "No Docstring" detection (most room for improvement)
   → Use semantic understanding to identify self-documenting code
   → Learn context-dependent documentation requirements
   → Target: >85% F1 on "No Docstring" task
""")

## Summary Statistics

In [ ]:
print("\n\nERROR ANALYSIS SUMMARY")
print("="*70)
print("\nConfusion Matrix Summary:")
print(confusion_df[['smell', 'fp', 'fn', 'fpr', 'fnr']].to_string(index=False))

print("\n\nBoundary Case Summary:")
print(boundary_df.to_string(index=False))

print("\n✅ Error analysis complete!")
print("📊 Key finding: Focus CodeBERT efforts on 'has_no_docstring' detection")